In [1]:
import time
import pandas as pd

In [2]:
rdd_time=[]
dataframe_time=[]

In [3]:
pip install pyspark

In [4]:
pip install openpyxl

# RDD

In [5]:
from pyspark.sql import SparkSession
spark:SparkSession = SparkSession.builder\
    .master("local[1]")\
    .appName("RDD Data")\
    .getOrCreate()    

In [6]:
start_time = time.time()
df =pd.read_excel('../input/anz-data/Worksheet in Forage - BIGDATA_Content Task.xlsx')
df1=spark.createDataFrame(df.astype(str))
t= (time.time() - start_time)
rdd_time.append(t)
print("--- %s seconds ---" %t)

In [7]:
df1.printSchema()

In [8]:
df1.show(5)

In [9]:
from pyspark.sql.functions import col
start_time = time.time()
df1=df1.filter((col('status')=='authorized') & (col('card_present_flag')==0.0))
t= (time.time() - start_time)
rdd_time.append(t)
print("--- %s seconds ---" %t)

In [10]:
from pyspark.sql.functions import split
start_time = time.time()

split_col1 = split(df1['long_lat'], '-')
split_col2 = split(df1['merchant_long_lat'], '-')
df2=df1.withColumn("long",split_col1.getItem(0)).\
withColumn("lat",split_col1.getItem(1)).\
withColumn("merch_long",split_col2.getItem(0)).\
withColumn("merch_lat",split_col2.getItem(1)).\
drop(*('long_lat','merchant_long_lat'))

t= (time.time() - start_time)
rdd_time.append(t)
print("--- %s seconds ---" %t)

In [11]:
start_time = time.time()
df2.write.option("header","true").csv("./RDD_output_file")
t= (time.time() - start_time)
rdd_time.append(t)
print("--- %s seconds ---" %t)

In [12]:
df2.show(3)

# Dataframe API

In [13]:
start_time = time.time()
spark1 = SparkSession.builder.appName('Dataframe').getOrCreate()
cols=df.columns
df1=spark1.createDataFrame(df.astype(str)).toDF(*cols)
t= (time.time() - start_time)
dataframe_time.append(t)
print("--- %s seconds ---" %t)

In [14]:
start_time = time.time()
df1=df1.filter((col('status')=='authorized') & (col('card_present_flag')==0.0))
t= (time.time() - start_time)
dataframe_time.append(t)
print("--- %s seconds ---" %t)

In [15]:
start_time = time.time()

split_col1 = split(df1['long_lat'], '-')
split_col2 = split(df1['merchant_long_lat'], '-')
df2=df1.withColumn("long",split_col1.getItem(0)).\
withColumn("lat",split_col1.getItem(1)).\
withColumn("merch_long",split_col2.getItem(0)).\
withColumn("merch_lat",split_col2.getItem(1)).\
drop(*('long_lat','merchant_long_lat'))

t= (time.time() - start_time)
dataframe_time.append(t)
print("--- %s seconds ---" %t)

In [16]:
start_time = time.time()
df2.write.option("header","true").csv("./Dataframe_output_file")
t= (time.time() - start_time)
dataframe_time.append(t)
print("--- %s seconds ---" %t)

## Comparing times

In [17]:
print('the time for reading, selecting relevant entires, splitting the columns and writing the file are \n')
print('For RDD')
print(rdd_time)
print('\nFor Dataframe')
print(dataframe_time)